<a href="https://colab.research.google.com/github/Fis428/PC03/blob/main/HW02_%E4%BD%9C%E6%A5%AD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HW02 Web Crawler

## 題目

*2023高人指點微醺地圖爬蟲*

## 摘要

本報告透過網路爬蟲技術，從高雄旅遊網「2023高人指點微醺地圖」[活動頁](https://khh.travel/zh-tw/event/news/4777/)
擷取以下資訊：
*   活動標題
*   更新日期
*   人氣
*   各行政區餐酒館清單（區域名稱、店名、地址、電話）


並將結果存成 CSV 檔，提供後續旅遊規劃及資料分析之用。



## 引言

隨著深度旅遊與在地飲食文化崛起，遊客對特色餐酒館的需求日益增加。高雄旅遊網官方活動頁整理了全市各區的酒吧與餐酒館資訊，包含地址與聯絡方式。若能自動化擷取並整合這些資料，不僅能節省手動蒐集時間，也能快速進行地理分布與人氣分析，對旅遊規劃具體參考價值。



## 方法


工具與技術
- 程式語言：Python 3.x

- 核心套件：

- `requests`：發送 HTTP 請求

- `BeautifulSoup`：解析 HTML

- `csv`：匯出資料

實作流程
1. 使用 `requests.get()` 取得頁面原始碼，設定 `resp.encoding = 'utf-8'` 以支援中文。

2. 以 `BeautifulSoup(resp.text, "html.parser")` 解析 HTML。

3. `.find()` 定位並擷取標題、更新日期、人氣三個元素的文字。

4. 以 `.find_all("h3")` 取得所有區域標題，並對每個 <h3> 執行` .find_next_siblings()`，收集同區之店家資訊。

5. 清理並拆解每段文字，分別擷取「店名」、「地址」（去除前綴「地址：」）與「電話」（去除「電話：」）。

6. 將整理後的字典列表寫入 CSV，欄位包含 Metadata（標題、更新日期、人氣）及資料列（Area／Name／Address／Tel）。






In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import re

url  = "https://khh.travel/zh-tw/event/news/4777/"
resp = requests.get(url)
resp.encoding = 'utf-8'
soup = BeautifulSoup(resp.text, "html.parser")

# 1. 標題：取第一個 <h2>
title = soup.find("h2").get_text(strip=True)

# 2. 更新日期：用 string= 正規比對「更新日期:YYYY-MM-DD」
date_line = soup.find(
    string=re.compile(r"更新日期 *: *\d{4}-\d{2}-\d{2}")
)
if date_line:
    date = re.search(r"\d{4}-\d{2}-\d{2}", date_line).group()
else:
    date = "N/A"

# 3. 人氣：先找到「人氣」文字，接著 find_next 找出第一個數字字串
pop_label = soup.find(string=re.compile(r"人氣"))
if pop_label:
    pop = pop_label.find_next(
        string=re.compile(r"\d+")
    ).strip()
else:
    pop = "0"

print("■ 標題：",    title)  # 2023高人指點微醺地圖
print("■ 更新日期：", date)   # 2024-05-02
print("■ 人氣：",    pop)     # 13110

# 4. 各區餐酒館列表（同之前邏輯）
data = []
for sec in soup.find_all("h3"):
    area = sec.get_text(strip=True)
    for sib in sec.find_next_siblings():
        if sib.name in ("h2","h3"):
            break
        if sib.name == "p":
            lines = sib.get_text("\n", strip=True).splitlines()
            name = lines[0]
            addr = next((l.replace("地址：","") for l in lines if l.startswith("地址：")), "")
            tel  = next((l.replace("電話：","") for l in lines if l.startswith("電話：")), "")
            data.append({
                "Area":    area,
                "Name":    name,
                "Address": addr,
                "Tel":     tel
            })

# 5. 匯出 CSV
with open("kaohsiung_nightmap.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["報名標題",  title])
    writer.writerow(["更新日期",  date])
    writer.writerow(["人氣",     pop])
    writer.writerow([])
    writer.writerow(["Area","Name","Address","Tel"])
    for row in data:
        writer.writerow([row["Area"], row["Name"], row["Address"], row["Tel"]])

print("CSV 已寫入：kaohsiung_nightmap.csv")


■ 標題： 2023高人指點微醺地圖
■ 更新日期： N/A
■ 人氣： 13114
CSV 已寫入：kaohsiung_nightmap.csv


## 結果


數據存儲方式

將擷取到的活動頁 Metadata（標題／更新日期／人氣）與各區店家清單儲存為 CSV 檔，檔名 kaohsiung_nightmap.csv，方便後續匯入 Excel 或 Pandas 做進一步分析。

In [ ]:
# 以 CSV Writer 將結果依序寫入：
with open("kaohsiung_nightmap.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    # metadata
    writer.writerow(["報名標題",  title])
    writer.writerow(["更新日期",  date])
    writer.writerow(["人氣",     pop])
    writer.writerow([])
    # header + data rows
    writer.writerow(["Area","Name","Address","Tel"])
    for row in data:
        writer.writerow([
            row["Area"],
            row["Name"],
            row["Address"],
            row["Tel"]
        ])


## 問題與挑戰

1.  HTML 標籤不一

 有時店家資訊可能不是以` <p> `呈現，改用 `<div>` 或 `<li>`，必須事先在 DevTools 檢查並調整 selector。
2. 文字節點 NoneType

  直接用` find(string=…)` 抓「更新日期」或「人氣」時，若匹配不到會回傳 `None`，必須加上  `if … is None` 判斷以免出錯。

3. 已棄用參數警告

  BeautifulSoup 原本的 `find(text=…)` 會跳 deprecation warning，改為 `find(string=…)` 即可。

4. 動態內容

  若未來碰到需要 AJAX 載入的資料，就必須改用 Selenium 或反向工程 API 端點。

## 結論


- 成功自動化擷取高雄旅遊網「2023高人指點微醺地圖」活動頁的所有關鍵資訊，包括標題、更新日期、人氣，以及依行政區分類的餐酒館名單。

- 將結果結構化為 CSV，減少手動整理時間，並能快速讓旅遊規劃者或資料分析師做後續統計、視覺化等應用。

- 未來可進一步延伸：

  - 多頁批次爬取：一次抓取高雄旅遊網多個活動或新聞頁面，累積更大樣本。

  - 地圖標註：結合 Google Maps API 或 Folium，將店家以地理座標顯示在互動地圖上。

  - 情感／熱度分析：若有評論文字，可做文字探勘，分析顧客評價與店家人氣關係。



## 參考文獻

* [高雄旅遊網「2023高人指點微醺地圖」](https://khh.travel/zh-tw/event/news/4777/)